# Focal mechanism calculation using `SKHASH` file format

**Make P-polarity input file i.e., `pol_consensus.csv` file.**

columns:\
        `event_id,event_id2,station,location,channel,p_polarity,origin_latitude,origin_longitude,origin_depth_km`



In [18]:
def make_skhash_polarity_file(data_path, catalog_path, output_path):
    
    """
    This function creates a polarity file for SKHASH format input data
    input: 
        data_path: path to the data folder that includes event folders i.e., folder names are event_ids
        and picks files from pyrocko..
        catalog_path: path to the catalog file
    output:
        pol_consensus.csv file written to ouput folder
    """

    import os
    import glob
    import pandas as pd
    import numpy as np

    # create an empty dataframe with column names [event_id,event_id2,station,location,channel,p_polarity,origin_latitude,origin_longitude,origin_depth_km]
    pol_df = pd.DataFrame()

    # get a list of folderes only in the data_path i.e. event_ids
    event_ids = [os.path.basename(path) for path in glob.glob(f"{data_path}/*") if os.path.isdir(path)]

    # loop over each event_id
    for i, event_id in enumerate(event_ids):

        # read the even_eventid_picks.csv file
        # only column columns 4,8,9
        # col4 has NN.SSSSS.LL.CCC format (network.station.location.channel)

        # check if polarity file has been created for this event_id
        if not os.path.exists(f"{data_path}/{event_id}/event_{event_id}_picks.txt"):
            print(f"{'='*10} Skipping event_id {event_id} as polarity file does not exist")
            continue

        picks_df = pd.read_csv(
            f"{data_path}/{event_id}/event_{event_id}_picks.txt",
            skiprows=1,         # skip the first row
            sep='\s+',          # delimiter
            header=None,        # no header
            usecols=[4,8,9],    # only read columns 4,8,9,
            names=['stns', 'phase', 'polarity']
            )
        # get 'origin_latitude','origin_longitude','origin_depth_km' from original catalog file
        ## or we can also use best_location file we created using misfit calculation
        catalog_df = pd.read_csv(catalog_path, sep=',', header=0, 
                                usecols=['latitude','longitude','depth', 'id'],
                                )
        catalog_df = catalog_df[catalog_df['id'] == event_id]

        # create a temporary dataframe with columns [event_id,event_id2,station,location,channel,p_polarity]
        temp_df = pd.DataFrame(columns=['event_id','event_id2','station','location','channel','p_polarity'])
        
        temp_df['station'] = picks_df['stns'].str.split('.').str[1]
        temp_df['location'] = picks_df['stns'].str.split('.').str[2]
        temp_df['location'].replace('', '--', inplace=True)

        temp_df['channel'] = picks_df['stns'].str.split('.').str[3]
        temp_df['p_polarity'] = picks_df['polarity']
        temp_df['event_id'] = event_id
        temp_df['event_id2'] = i+1
        temp_df['origin_latitude'] = format(catalog_df['latitude'].values[0], '.5f')
        temp_df['origin_longitude'] = format(catalog_df['longitude'].values[0], '.5f')
        temp_df['origin_depth_km'] = catalog_df['depth'].values[0]

        # append the temp_df to pol_df
        pol_df = pd.concat([pol_df, temp_df], ignore_index=True)

        # drop rows with nan values in p_polarity column
        pol_df.dropna(subset=['p_polarity'], inplace=True)
        

    # write the pol_df dataframe to a csv file
    print(f"{'='*10} Writing the `pol_consensus.csv` file to {output_path}")
    pol_df.to_csv(f'{output_path}/pol_consensus.csv', index=False)


# Make the station file
columns: 
- `station,location,channel,latitude,longitude,elevation` \
for this we will read the event_inventory.txt file.

In [19]:
def make_skhash_station_file(
    output_path,
    client_list = ['IRIS', 'NCEDC'], 
    starttime = "2008-01-01", 
    endtime = "2023-01-01",
    region = [-128, -122.5, 39, 42],
    channel = 'HH*,BH*,HN*,EH*',
    given_inventory = None
    ):

    """ 
    This function creates a station file for SKHASH format input data.
    No need to input a inventory file it will download the inventory from the client_list
    input:
        output_path: path to the output folder (mandatory)
        client_list: list of clients to download the inventory from (default: ['IRIS', 'NCEDC'])
        starttime: start time of the inventory (default: "2008-01-01")
        endtime: end time of the inventory (default: "2023-01-01")
        region: region of the inventory (default: [-128, -122.5, 39, 42])
        channel: channel of the inventory (default: 'HH*,BH*,HN*,EH*')
        given_inventory: path to the inventory file (default: None)
    output:
        station.csv file written to ouput folder
        
    """

    import obspy
    from obspy import read, UTCDateTime, read_inventory, Inventory
    from obspy.clients.fdsn import Client
    import pandas as pd
    import numpy as np
    import os
    
    if given_inventory == None:    
        # get station inventory

        starttime = UTCDateTime(starttime)
        endtime = UTCDateTime(endtime)

        minlongitude = region[0]
        maxlongitude = region[1]
        minlatitude = region[2]
        maxlatitude = region[3]

        merged_inventory = Inventory()
        for iclient in client_list:
            print(iclient)
            client = Client(iclient)
            try:
                try:
                    inv = client.get_stations(
                        starttime = starttime,
                        endtime = endtime,
                        minlatitude = minlatitude,
                        maxlatitude = maxlatitude,
                        minlongitude = minlongitude,
                        maxlongitude = maxlongitude,
                        channel = channel,
                        level = 'response'
                    )
                    merged_inventory.networks.extend(inv.networks)
                except:
                    pass
            except:
                print(f"Failed to get inventory from {iclient}")
                continue

        # temorarily write merged inventory to a file
        merged_inventory.write("mtj_merged_stations_temp.txt", format="STATIONTXT")
        given_inventory = "mtj_merged_stations_temp.txt"

    else:
        print(f"Using the provided merged inventory file: {given_inventory}")
        pass

    # read the merged inventory file
    inv_df = pd.read_csv(given_inventory, sep='|', skiprows=1,
                        usecols=[1, 2, 3, 4, 5, 6],
                        names=['station','location','channel','latitude','longitude','elevation'],
                        dtype={'station': str,'location': str, 'channel': str, 'latitude': float, 'longitude': float,
                                'elevation': float},    
                        )
    # replace the empty location with '--'
    # inv_df['location'].fillna('--', inplace=True)
    inv_df['location'].replace(pd.NA, '--', inplace=True)

    # sort by station and write the station file
    print(f"{'='*10} Writing the `station.csv` file to {output_path}")
    inv_df.sort_values(by=['station']).reset_index(drop=True)
    inv_df.drop_duplicates(subset=['station', 'location', 'channel'], keep='first', inplace=True) 
    inv_df.to_csv(f'{output_path}/station.csv', index=False)

    # delete the temporary file
    if os.path.exists("mtj_merged_stations_temp.txt"):
        os.remove("mtj_merged_stations_temp.txt")

    

# Make the reverse file
name: `reverse.csv` \
columns: `station,location,channel,start_time,end_time`

In [20]:
def make_skhash_reverse_file(polarity_file, output_path):
    """
    make sure the polarity_file has been created using the make_skhash_polarity_file function
    """
    import pandas as pd
    import numpy as np
    
    df = pd.read_csv(polarity_file, header=0)
    
    reverse_df = pd.DataFrame()
    reverse_df['station'], reverse_df['location'], reverse_df['channel'] = df['station'], df['location'], df['channel']
    reverse_df.drop_duplicates(subset=['station'], inplace=True)

    # put dummy start and end time
    reverse_df['start_time'] = "1900-01-01"
    reverse_df['end_time'] = "2200-01-01"

    print(f"{'='*10} Writing the `reverse.txt` file to {output_path}")
    reverse_df.to_csv(f'{output_path}/reverse.csv', index=False)

# Run the functions

In [21]:
# data_path = f"./examples/maacama_SKHASH_MTJ/IN/"
final_project_folder = "../../data"
data_path = f"{final_project_folder}/eq_data"
catalog_path = f"{final_project_folder}/above_slab_eq_0.2_grid.csv"
output_path = f"./examples/maacama_SKHASH_MTJ/IN/"

print(f"{data_path}")

make_skhash_polarity_file(data_path, catalog_path, output_path)

make_skhash_station_file(output_path, given_inventory = './examples/maacama_SKHASH_MTJ/IN/mtj_merged_stations.txt')

make_skhash_reverse_file(f"{output_path}/pol_consensus.csv", output_path)

f = "../../data/eq_data/"

../../data/eq_data
========== Writing the `pol_consensus.csv` file to ./examples/maacama_SKHASH_MTJ/IN/
Using the provided merged inventory file: ./examples/maacama_SKHASH_MTJ/IN/mtj_merged_stations.txt
========== Writing the `station.csv` file to ./examples/maacama_SKHASH_MTJ/IN/
========== Writing the `reverse.txt` file to ./examples/maacama_SKHASH_MTJ/IN/


In [22]:
import os

os.system('python3 SKHASH.py examples/maacama_SKHASH_MTJ/control_file.txt')

controlfile : examples/maacama_SKHASH_MTJ/control_file.txt
SKHASH v0.1 (2023-12-15)
Control file: examples/maacama_SKHASH_MTJ/control_file.txt
Creating lookup table (0/0): examples/velocity_models_MTJ/vz.MTJ.txt
	Created table.
Computing mechanisms in serial...
0 / 17	(nc40216664)
	S: 303.4656   D: 48.5403   R: -72.4056   U: 28.3596   Q: C
	Runtime: 0.17 sec
1 / 17	(nc51207076)
	Maximum azimuthal gap (150.498) > max_agap (90.0). Skipping.
2 / 17	(nc71100926)
	Maximum azimuthal gap (128.65) > max_agap (90.0). Skipping.
3 / 17	(nc71349716)
	S: 129.6605   D: 76.8136   R: 179.882   U: 41.8478   Q: D
	Runtime: 0.11 sec
4 / 17	(nc71544046)
	Maximum azimuthal gap (268.917) > max_agap (90.0). Skipping.
5 / 17	(nc71872550)
	Maximum azimuthal gap (99.495) > max_agap (90.0). Skipping.
6 / 17	(nc72006040)
	Maximum azimuthal gap (101.864) > max_agap (90.0). Skipping.
7 / 17	(nc72086051)
	Maximum azimuthal gap (262.12) > max_agap (90.0). Skipping.
8 / 17	(nc72322146)
	S: 126.7955   D: 59.7598   R: -

0